In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
os.environ["CC"] = "/opt/rh/devtoolset-10/root/usr/bin/gcc"
os.environ["CXX"] = "/opt/rh/devtoolset-10/root/usr/bin/g++"

In [ ]:
import nest_asyncio
nest_asyncio.apply()

import stan
import numpy as np

from mmur import IndependentHMConfusionMatrix
from mmur import DirichletMultinomialMultiConfusionMatrix
from mmur import Diric

In [ ]:
K = 1000 # n_obs
probs = [0.1,0.2,0.5,0.2]
N = 100 # n_cms
gam_a = 0.001
gam_b = 0.001

y= np.random.multinomial(K,probs,N)

In [ ]:
ind_hm = IndependentHMConfusionMatrix(1)
out_hm = ind_hm.fit_predict(y)

In [ ]:
dir_multi_mn = DirichletMultinomialMultiConfusionMatrix(1)
out_dir_multi = dir_multi_mn.fit_predict(y)

In [ ]:
probs_dm = out_dir_multi/1000
probs_ind_hm = out_hm/1000

In [ ]:
fit['pi_hat'].mean(axis=1) # 
probs = [0.1,0.2,0.5,0.2]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [ ]:
fit['y_hat'].sum(axis=0)

In [ ]:
d=0
fig,ax = plt.subplots(figsize = (14, 7))
sns.kdeplot(probs_dm[:,d],ax=ax, label = 'Dirichlet-Multinomial')
sns.kdeplot(probs_ind_hm[:,d],ax=ax, label = 'Independent Hierarchical')
plt.title(f'd={d}')
plt.legend()

In [ ]:
d=1
fig,ax = plt.subplots(figsize = (14, 7))
sns.kdeplot(probs_dm[:,d],ax=ax, label = 'Dirichlet-Multinomial')
sns.kdeplot(probs_ind_hm[:,d],ax=ax, label = 'Independent Hierarchical')
plt.title(f'd={d}')
plt.legend()

In [ ]:
d=2
fig,ax = plt.subplots(figsize = (14, 7))
sns.kdeplot(probs_dm[:,d],ax=ax, label = 'Dirichlet-Multinomial')
sns.kdeplot(probs_ind_hm[:,d],ax=ax, label = 'Independent Hierarchical')
plt.title(f'd={d}')
plt.legend()

In [ ]:
d=3
fig,ax = plt.subplots(figsize = (14, 7))
sns.kdeplot(probs_dm[:,d],ax=ax, label = 'Dirichlet-Multinomial')
sns.kdeplot(probs_ind_hm[:,d],ax=ax, label = 'Independent Hierarchical')
plt.title(f'd={d}')
plt.legend()

In [ ]:
d = 0
fig, ax = plt.subplots(figsize=(14, 7))
sns.kdeplot(fit['pi_hat'][d, :], ax=ax)
ax.axvline(x=probs[d], label='probability')
ax.axvline(x=(y/K).mean(axis=0)[d], label='y_proba',ls='--')
ax.legend()

In [ ]:
(y/K).mean(axis=0)

In [ ]:
d = 1
fig, ax = plt.subplots(figsize=(14, 7))
sns.kdeplot(fit['pi_hat'][d, :], ax=ax)
ax.axvline(x=probs[d], label='probability')
ax.axvline(x=(y/K).mean(axis=0)[d], label='y_proba',ls='--')
ax.legend()

In [ ]:
d = 2
fig, ax = plt.subplots(figsize=(14, 7))
sns.kdeplot(fit['pi_hat'][d, :], ax=ax)
ax.axvline(x=probs[d], label='probability')
ax.axvline(x=(y/K).mean(axis=0)[d], label='y_proba',ls='--')
ax.legend()

In [ ]:
d = 3
fig, ax = plt.subplots(figsize=(14, 7))
sns.kdeplot(fit['pi_hat'][d, :], ax=ax)
ax.axvline(x=probs[d], label='probability')
ax.axvline(x=(y/K).mean(axis=0)[d], label='y_proba',ls='--')
ax.legend()

In [ ]:
fig, ax = plt.subplots(figsize=(14, 7))
sns.kdeplot(fit['gam'].flatten(), ax=ax, label='post')
sns.kdeplot(np.random.gamma(gam_a, gam_b, size=fit['gam'].size), label='prior')
ax.legend()